In [84]:
import aiohttp
from yarl import URL
import asyncio
from lxml import etree
from pyquery import PyQuery as pq
import json

In [2]:
class Credential:
    def __init__(self, username, password):
        self.username = username
        self.password = password

    async def login(self, session: aiohttp.ClientSession):
        async with session.post(
            "https://apps.gwinnett.k12.ga.us/pkmslogin.form",
            data={
                "username": self.username,
                "password": self.password,
                "forgotpass": "p0/IZ7_3AM0I440J8GF30AIL6LB453082=CZ6_3AM0I440J8GF30AIL6LB4530G6=LA0=OC=Eaction!ResetPasswd==/#Z7_3AM0I440J8GF30AIL6LB453082",
                "login-form-type": "pwd",
            },
        ) as resp:
            if not resp.ok:
                raise RuntimeError("Login failed.")
            return resp

session = aiohttp.ClientSession()
student = Credential("202016378", "202016378")
login = await student.login(session)

In [73]:
async with session.get('https://apps.gwinnett.k12.ga.us/dca/student/dashboard') as resp:
    assert resp.ok
    url = pq(await resp.text())('#apps li:nth-child(1) > a').attr('href')

In [74]:
async with session.get(url) as resp:
    assert resp.ok
    text = await resp.text()

tree = etree.HTML(text.encode('utf-8'))

In [75]:
params = dict(zip(*[tree.xpath("//input/@%s" % i) for i in ["name", "value"]]))
url = tree.xpath('//form/@action')[0]
async with session.post(url, data=params) as resp:
    assert resp.ok
    text = await resp.text()
    url = resp.url


In [76]:
import functools
from typing import Any
import pyparsing as pp
import pyparsing.common as ppc


class JSParser:
    pp.ParserElement.enablePackrat()
    expr = pp.Forward()

    identifier = pp.Regex(r"[a-zA-Z_][a-zA-Z0-9_]*")
    number = ppc.number
    string = pp.quoted_string.set_parse_action(pp.removeQuotes)
    bool = pp.one_of("true false").set_parse_action(lambda s, l, t: t[0] == "true")
    array = pp.Group(
        pp.Literal("[").suppress()
        + (expr + pp.Literal(",")[0, 1].suppress())[...]
        + pp.Literal("]").suppress()
    ).set_parse_action(lambda s, l, t: t.as_list())
    hashable = string | number | bool | identifier
    map = (
        pp.Literal("{").suppress()
        + pp.Dict(
            pp.Group(
                hashable
                + pp.Literal(":").suppress()
                + expr
                + pp.Literal(",")[0, 1].suppress()
            )[...]
        )
        + pp.Literal("}").suppress()
    ).set_parse_action(lambda s, l, t: t.as_dict())
    null = pp.Literal("null").set_parse_action(lambda s, l, t: None)
    # A.B dot access
    access = pp.Combine(identifier + ("." + identifier)[1, ...])
    # enclosure
    enclosure = pp.Group(
        pp.Literal("(").suppress() + expr + pp.Literal(")").suppress()
    ).set_parse_action(lambda s, l, t: t[0])
    # call
    callee = access | identifier | enclosure
    call = callee("callee") + pp.Group(
        pp.Literal("(").suppress()
        + (expr + pp.Literal(",")[0, 1].suppress())[...]
        + pp.Literal(")").suppress()
    ).set_parse_action(lambda s, l, t: t.as_list())("args")
    # atom
    atom = (
        call
        | access
        | map
        | array
        | string
        | bool
        | null
        | identifier
        | number
        | enclosure
    )
    # common binary operators
    bin_expr = pp.Group(
        atom + (pp.one_of("&& || == === != !== > >= < <= + - * / %") + atom)[1, ...],
        aslist=True,
    )
    expr << (bin_expr | atom)

    assignable = access | identifier
    assign = pp.Group(assignable + (pp.Suppress("=") + expr)[1, ...])

    var_decl = pp.Group(
        pp.Optional(pp.one_of("var", "let"))
        + pp.OneOrMore(assign + pp.Literal(",")[0, 1].suppress())("assigns")
        + (pp.Literal(";") | pp.LineEnd())
    )
    expr_stmt = pp.Group(expr + (pp.Literal(";") | pp.LineEnd()))
    stmt = expr_stmt | var_decl
    prog = pp.ZeroOrMore(stmt)

    def __init__(self, js: str):
        self.js = js

    @functools.cached_property
    def variables(self) -> dict[Any, Any]:
        """
        Parse javascript code and return a dict of variables in the code.

        Notice that this function only supports simple javascript code, i.e.,
        only variable declaration and some simple expressions are supported.
        """

        variables = {}
        for result in self.prog.parse_string(self.js):
            if not hasattr(result, "assigns"):
                continue
            for assign in result.assigns:
                for i in range(len(assign) - 1):
                    variables[assign[i]] = assign[-1]
        return variables

    def __getitem__(self, name: str) -> Any:
        return self.variables[name]


In [77]:
tree = etree.HTML(text.encode('utf-8'))
js = tree.xpath('//script/text()')[0]
parser = JSParser(js)
items = parser['PXP.NavigationData']['items']
grade_book = next(filter(lambda x: x['description'] == 'Grade Book', items))

async with session.get(url.join(URL(grade_book['url']))) as resp:
    assert resp.ok
    text = await resp.text()
    url = resp.url

In [78]:
tree = etree.HTML(text.encode('utf-8'))
js = tree.xpath('//script/text()')[0]
parser = JSParser(js)

In [79]:
parser = JSParser(pq(text)("script")[0].xpath("./text()")[0])

school = parser["PXP.GBFocusData"]["Schools"][0]
agu = parser["PXP.AGU"]
periods = [period for period in school["GradingPeriods"] if period["defaultFocus"]]

async def _post(path, data, *, xhr=True):
    headers = {"CURRENT_WEB_PORTAL": "StudentVUE"}
    if xhr:
        headers["X-Requested-With"] = "XMLHttpRequest"
    async with session.post(
        url.join(URL(path)),
        json=data,
        headers=headers,
    ) as resp:
        assert resp.ok
        return await resp.json()

async def load_control(session, name, params):
    data = await _post(
        "service/PXP2Communication.asmx/LoadControl",
        {"request": {"control": name, "parameters": params}},
    )
    if error := data["d"]["Error"]:
        raise Exception(error)
    return data["d"]["Data"]


async def _fetch_all_class(session, period: dict[str, str]):
    school_id = period["schoolID"]
    grading_period_group = period["GroupName"]
    grade_period_gu = period["GU"]
    org_year_gu = period["OrgYearGU"]

    params = {
        "gradePeriodGU": grade_period_gu,
        "GradingPeriodGroup": grading_period_group,
        "schoolID": school_id,
        "OrgYearGU": org_year_gu,
        "AGU": agu,
    }
    data = await load_control(session, "Gradebook_SchoolClasses", params)
    html = data["html"]

    tree = etree.HTML(data[0][1].encode('utf-8'))
    table = tree.xpath('//div[contains(@class, "table")]')[0]
    table.remove(table.xpath('//div[contains(@class, "table-header")]')[0])
    rows = [
        json.loads(row)
        for row in table.xpath(
            './div/div[contains(@class, "row") and contains(@class, "header")]//button/@data-focus'
        )
    ]


async def fetch(session, periods: list[dict[str, str]]):
    return await asyncio.gather(*[_fetch_all_class(session, period) for period in periods])


data = await fetch(session, periods)


In [81]:
async def choose_class(session, cls):
    data = {
        "request": {
            "control": cls['LoadParams']['ControlName'],
            "parameters": cls['FocusArgs'],
        }
    }
    await _post("service/PXP2Communication.asmx/LoadControl", data)

[{'LoadParams': {'ControlName': 'Gradebook_RichContentClassDetails',
   'HideHeader': False},
  'FocusArgs': {'viewName': None,
   'studentGU': 'B2B4330C-5388-4E9E-9A8B-B886A4FBE89B',
   'schoolID': 136,
   'classID': 1331467,
   'markPeriodGU': 'CE6B4B57-FF4F-4F7D-961F-EFF787C0ADB7',
   'gradePeriodGU': '1534A1B6-9B06-4151-A883-22FAEDA8FBD0',
   'subjectID': -1,
   'teacherID': -1,
   'assignmentID': -1,
   'standardIdentifier': None,
   'AGU': '0',
   'OrgYearGU': '5461EE39-F6EA-44E9-9A06-61A8B9D51650',
   'gradingPeriodGroup': None}},
 {'LoadParams': {'ControlName': 'Gradebook_RichContentClassDetails',
   'HideHeader': False},
  'FocusArgs': {'viewName': None,
   'studentGU': 'B2B4330C-5388-4E9E-9A8B-B886A4FBE89B',
   'schoolID': 136,
   'classID': 1331119,
   'markPeriodGU': 'CE6B4B57-FF4F-4F7D-961F-EFF787C0ADB7',
   'gradePeriodGU': '1534A1B6-9B06-4151-A883-22FAEDA8FBD0',
   'subjectID': -1,
   'teacherID': -1,
   'assignmentID': -1,
   'standardIdentifier': None,
   'AGU': '0',
 

In [13]:
semester, html = data[0]
doc = pq(html)
rows = list(doc(".pres-table > div:nth-child(2) > div").items())
classes = list(
    zip(
        rows[::2],
        rows[1::2],
    )
)
[
    (
        float(mark) if (mark := body(".mark").text()) != "N/A" else -1,
        header(".course-title").text().split(": ", 1)[1].strip().title(),
    )
    for header, body in classes
]


[(99.0, 'Mstry Band Ii'),
 (99.0, 'Ap Cal Bc Gf'),
 (100.0, 'Span Ii Gf'),
 (94.0, 'Sci/ Eng/Res Gf'),
 (98.0, 'Ap Biology Gf'),
 (97.0, 'Adv Phys Robotic Gf'),
 (92.0, '10 Lit & Comp Gf'),
 (99.0, 'Ap Wor Hist Gf')]

In [26]:
def dump_json(data):
    import json

    with open("data.json", "w") as f:
        json.dump(data, f, indent=4)

In [82]:
data = {
    "request": {
        "control": "Gradebook_RichContentClassDetails",
        "parameters": {
            "schoolID": 136,
            "classID": 1331119,
            "gradePeriodGU": "1534A1B6-9B06-4151-A883-22FAEDA8FBD0",
            "subjectID": -1,
            "teacherID": -1,
            "markPeriodGU": "CE6B4B57-FF4F-4F7D-961F-EFF787C0ADB7",
            "assignmentID": -1,
            "standardIdentifier": None,
            "viewName": "courseContent",
            "studentGU": "B2B4330C-5388-4E9E-9A8B-B886A4FBE89B",
            "AGU": "0",
            "OrgYearGU": "5461EE39-F6EA-44E9-9A06-61A8B9D51650",
        },
    }
}
await _post("service/PXP2Communication.asmx/LoadControl", data)


{'d': {'__type': 'PXP.PXPInfo.PXPWebResponse',
  'Error': None,
  'Data': {'html': '\r\n\r\n<script type="text/javascript">\r\n    PXP.GBWI_Translation = {"changeGradeText":"Change the grade for this assignment to see how it impacts your class grade","classGradeText":"Class Grade Would Be:","percentageText":"Percentage Would Be:","resetButtonText":"RESET","youScoreText":"If You Score:","calcButton":"CALCULATE MAX SCORE","gridTooltip":"Open\xa0the\xa0\\"what\xa0if\\"\xa0calculator\xa0for\xa0this\xa0assignment.","calcText":"Calculate the best grade possible if all ungraded work was graded at full value.","gradePill":"Grade Could Be:"};\n\t\r\n</script>\r\n<script language="JavaScript" type="text/javascript">\nPXP.GBFocus.SetViewName(\'courseContent\');</script>\r\n\r\n<script type="text/javascript">CURRENT_WEB_PORTAL=\'StudentVUE\';</script><script type="text/javascript">applicationRoot=\'/\';</script>\r\n\r\n<div class="component-loader-container" data-bind="attr: { \'data-bound\': \'tr

In [72]:
data

{'request': {'control': 'Gradebook_RichContentClassDetails',
  'parameters': {'schoolID': 136,
   'classID': 1331119,
   'gradePeriodGU': '1534A1B6-9B06-4151-A883-22FAEDA8FBD0',
   'subjectID': -1,
   'teacherID': -1,
   'markPeriodGU': 'CE6B4B57-FF4F-4F7D-961F-EFF787C0ADB7',
   'assignmentID': -1,
   'standardIdentifier': None,
   'viewName': 'courseContent',
   'studentGU': 'B2B4330C-5388-4E9E-9A8B-B886A4FBE89B',
   'AGU': '0',
   'OrgYearGU': '5461EE39-F6EA-44E9-9A06-61A8B9D51650'}}}

{'d': {'__type': 'PXP.PXPInfo.PXPWebResponse',
  'Error': None,
  'Data': {'html': '\r\n\r\n<script type="text/javascript">\r\n    PXP.GBWI_Translation = {"changeGradeText":"Change the grade for this assignment to see how it impacts your class grade","classGradeText":"Class Grade Would Be:","percentageText":"Percentage Would Be:","resetButtonText":"RESET","youScoreText":"If You Score:","calcButton":"CALCULATE MAX SCORE","gridTooltip":"Open\xa0the\xa0\\"what\xa0if\\"\xa0calculator\xa0for\xa0this\xa0assignment.","calcText":"Calculate the best grade possible if all ungraded work was graded at full value.","gradePill":"Grade Could Be:"};\n\t\r\n</script>\r\n<script language="JavaScript" type="text/javascript">\nPXP.GBFocus.SetViewName(\'courseContent\');</script>\r\n\r\n<script type="text/javascript">CURRENT_WEB_PORTAL=\'StudentVUE\';</script><script type="text/javascript">applicationRoot=\'/\';</script>\r\n\r\n<div class="component-loader-container" data-bind="attr: { \'data-bound\': \'tr

In [36]:
# assignment meta data
data = await _post(
    "./api/GB/ClientSideData/Transfer?action=genericdata.classdata-GetClassData",
    {
        "FriendlyName": "genericdata.classdata",
        "Method": "GetClassData",
        "Parameters": "{}",
    },  
)
dump_json(data)


In [29]:
# random meta data, group fields, etc.
data = await _post(
    "./api/GB/ClientSideData/Transfer?action=pxp.course.content-get",
    {"FriendlyName": "pxp.course.content", "Method": "get", "Parameters": "{}"},
)
dump_json(data)

In [30]:
# missing nad upcoming
data = await _post(
    "./api/GB/ClientSideData/Transfer?action=pxp.course.cards-get",
    {"FriendlyName": "pxp.course.cards", "Method": "get", "Parameters": "{}"},
)
dump_json(data)

In [32]:
# all assignments
data = await _post(
    "./api/GB/ClientSideData/Transfer?action=pxp.course.content.items-LoadWithOptions",
    {
        "FriendlyName": "pxp.course.content.items",
        "Method": "LoadWithOptions",
        "Parameters": '{"loadOptions":{"sort":[{"selector":"due_date","desc":false}],"filter":[["isDone","=",false]],"group":[{"Selector":"Week","desc":false}],"requireTotalCount":true,"userData":{}},"clientState":{}}',
    },
)
dump_json(data)


In [33]:
# summary of class content and grade card
data = await _post(
    "./api/GB/ClientSideData/Transfer?action=pxp.course.grade.card-get",
    {"FriendlyName": "pxp.course.grade.card", "Method": "get", "Parameters": "{}"},
)
dump_json(data)

In [12]:
async with session.post(
    url.join(URL("./api/GB/ClientSideData/Transfer?action=pxp.course.content.items-LoadWithOptions")),
    json={
        "FriendlyName": "pxp.course.content.items",
        "Method": "LoadWithOptions",
        "Parameters": "{\"loadOptions\":{\"sort\":[{\"selector\":\"due_date\",\"desc\":false}],\"filter\":[[\"isDone\",\"=\",false]],\"group\":[{\"Selector\":\"Week\",\"desc\":false}],\"requireTotalCount\":true,\"userData\":{}},\"clientState\":{}}"
    },
    headers={
        "X-Requested-With": "XMLHttpRequest",
        "CURRENT_WEB_PORTAL": "StudentVUE",
    },
) as resp:
    data = await resp.json()

data

{'metaData': {'properties': {},
  'entity': {'allowAdding': False,
   'allowDeleting': False,
   'allowUpdating': False,
   'isVisible': True},
  'instance': {'allowDeleting': True,
   'allowUpdating': True,
   'isVisible': True},
  'extraData': {'teacherNames': [''], 'subjects': ['(No Subject)']}},
 'responseData': {'data': [{'key': 'Week 03 - 8/14/2022 through 8/20/2022',
    'items': [{'isDone': False,
      'id': 5,
      'xrefID': -1,
      'title': 'Pass-Off #1- G Major Scale',
      'week': 'Week 03 - 8/14/2022 through 8/20/2022',
      'unit': 'N/A',
      'subject': '(No Subject)',
      'subjectId': -1,
      'assignmentType': 'AKS Progress*',
      'due_date': '2022-08-19T00:00:00',
      'itemID': 15981774,
      'itemType': 'GradeBookItem',
      'date': 'August 19 - Friday',
      'pointsPossible': '100.00',
      'embeds': [],
      'isMissing': False,
      'gradeMark': '100',
      'showAssignmentGrade': True,
      'commentText': '',
      'calcValue': '100%',
      '

In [301]:
url.join(
        URL("api/GB/ClientSideData/Transfer?action=genericdata.classdata-GetClassData")
    )

URL('https://ga-gcps-psv.edupoint.com/api/GB/ClientSideData/Transfer?action=genericdata.classdata-GetClassData')